In [ ]:
! pip install -U spacy

In [ ]:
import os
import pprint
import string
import re
import pandas as pd
import plotly.express as px

import nltk
from nltk.text import Text
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.collocations import BigramCollocationFinder,BigramAssocMeasures
import spacy
from spacy import displacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('all')
pp = pprint.PrettyPrinter(indent=4, compact=True)
#nlp = spacy.load("es_core_news_lg")

# Stop words del español
english_stops = set(stopwords.words('english'))
english_stops.update(string.punctuation)

In [ ]:
descr_df = pd.read_csv('../input/ted-talks/ted_main.csv')
trans_df = pd.read_csv('../input/ted-talks/transcripts.csv')

In [ ]:
merged_df = pd.merge(descr_df, trans_df, on="url")

In [ ]:
merged_df.head()

In [10]:
def remove_stop_words(text):
  """
    Remueve stop words en español

    Attributes
    ----------
    text: list
      lista de palabras (tokens) a filtrar

    Returns
    -------
    list
      lista de palabras sin los stop words
  """
  return [token for token in text if token.lower() not in english_stops]

In [ ]:
def lematize_words(text):
  """
    Lematización de palabras - aplica lematización de palabras sobre un set de tokens

    Attributes
    ----------
    text: list
      lista de palabras (tokens) sobre los cuales se aplicará la lematización
    
    Returns
    -------
    list
      lista con todas las lematizaciones de las palabras
  """
  doc = nlp(" ".join(text))
  return [token.lemma_ for token in doc]

In [ ]:
def stem_words(text):
  """
    Stemming de palabras - aplica stemming de palabras sobre un set de tokens

    Attributes
    ----------
    text: list
      lista de palabras (tokens) sobre las que se aplicará stemming
    
    Returns
    -------
    list
      lista con los stems de las palabras
  """
  stemmer = SnowballStemmer('spanish')
  return [stemmer.stem(word) for word in text]

In [ ]:
def remove_meaningless_words(text):
  """
    Remueve palabras sin significado

    Attributes
    ----------
    text: list
      lista de palabras (tokens) a filtrar
    
    Returns
    -------
    list
      lista de palabras filtrada en base a expresiones regulares
  """
  patterns = [r"(^={1,}=$)", r'\u200b']
  tokens = text
  for pattern in patterns:
    regexp = re.compile(pattern)
    tokens = [token for token in tokens if not regexp.search(token)]
  return tokens

In [ ]:
def tokenize(text, mode='word'): 
  """
    Tokenización de documento - tokeniza un documento por palabra o por oración

    Attributes
    ----------
    text: str
      documento a tokenizar
    mode: str, optional
      método de tokenización (default: 'word' (por palabra))
    
    Returns
    -------
    list
      lista de tokens 
    
    Raises
    ------
      Exception
        si el mode no es 'word' o 'sentence'
  """
  if mode == 'word':
    return word_tokenize(text, language='english')
  elif mode == 'sentence':
    return sent_tokenize(text, language='english')
  else:
    raise Exception('metodo de tokenizacion no encontrado')

In [ ]:
def pre_procesamiento_texto(text):
  """
    Pre-procesamiento y obtención de las 20 palabras más significativas

    Attributes
    ----------
    text: str
      documento a analizar

    Returns
    -------
    pd.DataFrame
      retorna un dataframe con las 20 palabras que más se repiten y su frecuencia
  """
  tokenized = tokenize(text)
  without_stops = remove_stop_words(tokenized)
  meaningfull_tokens = remove_meaningless_words(without_stops)
  lematized_words = lematize_words(meaningfull_tokens)
  # stemmed_words = stem_words(lematized_words
  return lematized_words

results = pre_procesamiento_texto(merged_df.transcript)